# Import/Export a Feedback Dataset

To push `FeedbackTask` datasets to Argilla or pull them from Argilla, you can use the `FeedbackDataset.push_to_argilla()` and `FeedbackDataset.from_argilla()` methods, respectively. Additionally, you can also import and export datasets to HuggingFace for persistance and sharing, with the `FeedbackDataset.push_to_huggingface()` and `FeedbackDataset.from_huggingface()` methods, respectively.

In the following section, we'll assume that the variable `dataset` is an instance of `FeedbackDataset` that has been previously created locally. If you want to follow along, you can create a new dataset with the following code:

In [ ]:
from argilla import rg

dataset = rg.FeedbackDataset(
    fields=[
        rg.TextField(name="question"),
        rg.TextField(name="answer"),
    ],
    questions=[
        rg.RatingQuestion(
            name="rating", 
            title="Rate the quality of the response:", 
            description="1 = very bad - 5= very good",
            required=True,
            values=[1, 2, 3, 4, 5]
        ),
        rg.TextQuestion(
            name="corrected-text",
            title="Provide a correction to the response:",
            required=False
        )
    ]
)

## Cloud Storage

### Argilla

Once the `FeedbackDataset` is created locally (not pushed anywhere yet), we can start adding records to it with the method `add_records`, based on the `fields` and, if we have `responses` already, based on the `questions` too. Once we're happy with the dataset, we can push it to Argilla with the `FeedbackDataset.push_to_argilla()` method, so that we can see it in the UI.

In [ ]:
dataset.push_to_argilla(name="my-dataset", workspace="my-workspace")

Additionally, we can also pull a dataset from Argilla with the `FeedbackDataset.from_argilla()` method. This method will return a new instance of `FeedbackDataset` with the same guidelines, fields, questions, and records (including annotations if any) as the dataset in Argilla. And then we can push it again to Argilla if we want e.g. under a different name, if we included new records, etc.

In [ ]:
dataset = rg.FeedbackDataset.from_argilla("my-dataset", workspace="my-workspace")
dataset.push_to_argilla(name="my-dataset-clone", workspace="my-workspace")

### HuggingFace Hub

It is also possible to save and load `FeedbackDataset`s into the HuggingFace Hub for persistance. So on, the methods `push_to_huggingface` and `from_huggingface` allow you to push those to the HuggingFace Hub, or to pull those locally from the HuggingFace Hub, respectively.

Note that just the `FeedbackDataset`s that have been pushed to the HuggingFace Hub via `push_to_huggingface` can be loaded back with `from_huggingface`, as we're not just uploading the dataset records, but also a configuration file named `argilla.cfg`, that contains the dataset configuration i.e. the fields, questions, and guidelines, if any.

Also note that the args and kwargs of `push_to_huggingface` are the args of `push_to_hub` from 🤗[datasets](https://github.com/huggingface/datasets), and the ones of `from_huggingface` are the args of `load_dataset` from 🤗[datasets](https://github.com/huggingface/datasets).

In [ ]:
# Push to HuggingFace Hub
dataset.push_to_huggingface("argilla/my-dataset-clone")

# Push to HuggingFace Hub as private
dataset.push_to_huggingface("argilla/my-dataset-clone", private=True, token="...")

# Load a public dataset
dataset = rg.FeedbackDataset.from_huggingface("argilla/my-dataset")

# Load a private dataset
dataset = rg.FeedbackDataset.from_huggingface("argilla/my-dataset", use_auth_token=True)

Additionally, when pushing a `FeedbackDataset` to the HuggingFace Hub, one can provide the param `generate_card` to generate and push the `DatasetCard` too. `generate_card` default value is True, so it will always be generated unless `generate_card=False` is specified.

### Combination

Additionally, one could combine all the different cloud storage options to e.g. pull a `FeedbackDataset` from Argilla, and then push it to the HuggingFace Hub, or viceversa.

In [ ]:
# Pull from Argilla
dataset = rg.FeedbackDataset.from_argilla("my-dataset", workspace="my-workspace")

# Push to HuggingFace Hub
dataset.push_to_huggingface("argilla/my-dataset-clone")

## Local Storage

Additionally, due to the integration with 🤗`datasets`, as a workaround, you can also export the records of a `FeedbackDataset` locally in your preferred format by converting the `FeedbackDataset` to a `datasets.Dataset` first using the method `format_as("datasets")`, and then exporting the `datasets.Dataset` to either CSV, `pandas`, Parquet, etc.

Note that we are relying on 🤗`datasets` to not re-invent the wheel, but take into consideration that this workaround will just export the records into the desired format, not the dataset itself, so those cannot be loaded back automatically for the moment.

In [ ]:
hf_dataset = dataset.format_as("datasets")

hf_dataset.save_to_disk("sharegpt-prompt-rating-mini")  # Save as a `datasets.Dataset` in the local filesystem
hf_dataset.to_csv("sharegpt-prompt-rating-mini.csv")  # Save as CSV
hf_dataset.to_parquet()  # Save as Parquet

hf_dataset.to_pandas()  # Convert to `pandas.DataFrame`